In [53]:
#!/usr/bin/python3
'''A tkinter app that will take input values for
1.Building
2.Room
3.RoomOccupancy
4.CollectionOperator
and finds the values for
5.Collection Device
6.DateTime
7.Humidity
8.Temperature
and  creates a database to save and then export the database table into a csv
'''

#Author:Ashik Vittal Poovappa
#Date: 12/13/2018

import sqlite3
import csv
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import platform
import datetime
from tkinter import ttk
#from sense_hat import SenseHat

class main_GUI():
    def __init__(self,master):
        self.master=master
        master.title("Analysis")
        scroll=Scrollbar(master)
        
        
        '''Entry widget for input values'''
        
        labels=["Building Name","Room Number","Collection Operator", 'Room Occupancy']
        for i in labels:
            mainLabel=Label(master,text=i).grid(sticky=E+W)
        
        self.entry_text = StringVar()
        
        self.Building=ttk.Entry(textvariable =self.entry_text)
        self.Building.grid(row=0,column=1,sticky=E+W)
        
        self.Room=ttk.Entry()
        self.Room.grid(row=1,column=1,sticky=E+W)
        
        self.CollectionOperator=ttk.Entry()
        self.CollectionOperator.grid(row=2,column=1,sticky=E+W)
        
        '''Checkbox for room occupancy'''
        self.c1=tk.BooleanVar()
        self.c2=tk.BooleanVar()
        tk.Checkbutton(text='True',variable=self.c1).grid(row=3,column=1,sticky=E+W)
        tk.Checkbutton(text='False',variable=self.c2).grid(row=3,column=2,sticky=E+W)
       
        
        
        '''Creating database in memory mode'''
        self.createDb()
        
        
        '''Buttons'''
        resetButton=ttk.Button(master,text="Reset", command=lambda:self.resetEntry()).grid(row=6,column=0,padx=5,pady=5)
        dbButton=ttk.Button(master,text="Collect and Save Values", command=lambda:self.takeReading()).grid(row=6,column=1,padx=5,pady=5)
        exportButton=ttk.Button(master,text="Save as CSV", command=lambda:self.export()).grid(row=6,column=2,padx=5,pady=5)
        deleteAllButton=ttk.Button(master,text="Stop Data Collection", command=lambda:self.deleteAll()).grid(row=1,column=2,padx=5,pady=5)
        deleteAllButton=ttk.Button(master,text="EXIT", command=lambda:self.exit()).grid(row=10,column=1,padx=5,pady=5,sticky=N+S+E+W)

        
        
        '''creating listbox to display values'''
        listMain = Frame(master)
        listMain.grid(row=9, column=1, sticky="ns")
        self.displayList = Listbox(listMain, width=70, height=10,yscrollcommand=scroll.set)
        self.displayList.grid(row=7,column=0,pady=5)
        scroll.config(command=self.displayList.yview)
        
        
    '''Reset entry widget'''    
    def resetEntry(self):
        entryLables=[self.Building,self.Room,self.CollectionOperator]
        for x in entryLables:
            x.delete(0, 'end')
    
    
    def senseHumidity(self):
        sense = SenseHat()
        self.humidity = sense.get_humidity()
        sense.show_message(str(self.humidity))
        self.displayList.insert(END,'Humidity: {}'.format(self.humidity))
        
    def senseTemp(self):
        sense = SenseHat()
        self.temp = sense.get_temperature()
        sense.show_message(str(self.temp))
        self.displayList.insert(END,'Temperature: {}'.format(self.temp))
    
    '''function to collect all the values and display in listbox '''    
    def takeReading(self):
        if len(self.entry_text.get())== 0:
            self.displayList.insert(END,'All fields are Mandatory!!')
            
        buildingValue=self.Building.get()
        self.displayList.insert(END,'Building: {}'.format(buildingValue))
        
        roomValue=self.Room.get()
        self.displayList.insert(END,'Room: {}'.format(roomValue))
        
        if self.c1.get()==True:
            occupancyValue='True'
        else:
            occupancyValue='False'
        self.displayList.insert(END,'Occupancy: {}'.format(self.c1.get()))
        
        operatorValue=self.CollectionOperator.get()
        self.displayList.insert(END,'Operator Name: {}'.format(operatorValue))
        
        deviceName=platform.node()
        self.displayList.insert(END,'Device: {}'.format(deviceName))
        
        dateTime=datetime.datetime.now()
        self.displayList.insert(END,'Date and Time: {}'.format(dateTime))
        
        self.senseHumidity()
        self.senseTemp()
        
        self.tableValues = (buildingValue,roomValue,dateTime,occupancyValue,
                            operatorValue,deviceName,self.humidity,self.temp)
    
    
        c=self.conn.cursor()
        
        try:
            c.execute(''' INSERT INTO reading(Building,Room,DateTime,RoomOccupancy,CollectionOperator,ColleectionDevice,Humidity,Temperature)
                  VALUES(?,?,?,?,?,?,?,?) ''',(self.tableValues)) #insert to table
        except sqlite3.IntegrityError as e:
            self.displayList.insert(END,'Please select a Different Building Name')
        
  
    '''creating database'''
    def createDb(self):
        self.conn = sqlite3.connect(":memory:")
        self.conn.cursor()
        c =self.conn.cursor()
        if self.conn is not None:
            c.execute('''CREATE TABLE IF NOT EXISTS reading(
            Building TEXT PRIMARY KEY ,
            Room INTEGER NOT NULL,
            DateTime TEXT ,
            RoomOccupancy TEXT,
            CollectionOperator TEXT,
            ColleectionDevice TEXT,
            Humidity,
            Temperature);''')
        else:
            print("Error! cannot create the database connection.")
            
    '''Delete all readings'''        
    def deleteAll(self):
        
        '''Warning before deleting all data'''
        answer = messagebox.askyesno("Warning","Clicking Yes will Stop data Collection and delete all previous data! Continue?")
        
        if answer==True:
            c =self.conn.cursor()
            c.execute('DELETE FROM reading')
            self.displayList.delete(0,'end')
            self.resetEntry()
            self.displayList.insert(END,'All Data Deleted')
        else:
            pass 
        
    def exit(self):
        root.destroy()
        
    
       
    '''Select the table and export as csv'''
    def export(self):
        c =self.conn.cursor()
        c.execute("SELECT * FROM reading") #select all readings from table
        rows = c.fetchall()
        for row in rows:
            self.displayList.insert(END,format(row))    #verify if the values are present and correct
        path=filedialog.asksaveasfilename(defaultextension=".csv",initialfile='PartA', title="Save CSV")
        if path is None:
            self.displayList.insert(END,'Please select a Folder to save!')
        elif path is ():
            self.displayList.insert(END,'Please select a Folder to save!')
        elif path is '':
            self.displayList.insert(END,'Please select a Folder to save!')
        else:
            csvWriter = csv.writer(open(path, "w",newline=""))
            csvWriter.writerow(['Building','Room','DateTime','RoomOccupancy','CollectionOperator','ColleectionDevice','Humidity','Temperature'])
            csvWriter.writerows(rows)
            self.displayList.insert(END,'CSV Saved: {}'.format(path))
            
        
        
root=tk.Tk()
app=main_GUI(root)
root.mainloop()